# Statistical Arbitrage with Correlation Matrix Clustering
# Enhanced Example Notebook

## Introduction

This notebook demonstrates the implementation of the paper "Correlation Matrix Clustering for Statistical Arbitrage Portfolios" by Cartea, Cucuringu, and Jin (2023), with significant enhancements to the original methodology.

The approach consists of two main steps:
1. **Clustering**: Group stocks based on their residual return correlation matrix using graph clustering algorithms
2. **Portfolio Construction**: Build mean-reverting portfolios within each cluster

We'll go through the entire process step by step, from data preparation to portfolio evaluation, including all the enhanced features of our implementation.

## 1. Setup and Import Libraries

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100
sns.set_palette('tab10')

# Import our custom modules
from data_preprocessing import prepare_stock_data, fetch_stock_data, calculate_returns, prepare_multifactor_data
from correlation_matrix import compute_correlation_matrix, decompose_correlation_matrix
from clustering import (
    cluster_stocks, get_clusters_dict, determine_clusters_mp, determine_clusters_variance,
    hierarchical_clustering, density_based_clustering, deep_embedding_clustering
)
from portfolio_construction import (
    identify_winners_losers, construct_arbitrage_portfolios, calculate_portfolio_returns
)
from utils import (
    calculate_annualized_return,
    calculate_sharpe_ratio,
    calculate_sortino_ratio,
    plot_correlation_matrix,
    plot_clusters,
    get_eigenvalue_distribution
)
from visualization_plotly import (
    plot_cumulative_returns,
    plot_performance_metrics,
    plot_drawdowns,
    plot_monthly_returns_heatmap,
    plot_transaction_costs,
    create_performance_report
)
from backtest import (
    StatisticalArbitrageBacktest, run_industry_benchmark,
    time_series_cross_validation, monte_carlo_backtest, walk_forward_optimization
)
from fama_french import get_fama_french_industries, get_industry_names
from adaptive_parameters import AdaptiveParameterManager
from alternative_data import (
    SentimentDataProcessor, OptionsDataProcessor, OrderFlowProcessor
)
from portfolio_integration import PortfolioAllocator, calculate_information_ratio_impact

## 2. Data Preparation

We'll start by defining a universe of stocks and fetching historical price data. For this example, we'll use a diverse set of large-cap stocks from different sectors.

In [ ]:
# Define parameters
start_date = '2018-01-01'
end_date = '2022-12-31'
beta_window = 60  # Days for beta calculation

# Define a universe of stocks from different sectors
tickers = [
    # Technology
    'AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA', 'INTC', 'AMD', 'ADBE', 'CRM', 'CSCO',
    # Financial
    'JPM', 'BAC', 'GS', 'MS', 'V', 'MA', 'PYPL', 
    # Healthcare
    'JNJ', 'PFE', 'MRK', 'ABT', 'UNH',
    # Consumer
    'AMZN', 'WMT', 'PG', 'KO', 'MCD', 'NKE', 'SBUX',
    # Industrial
    'GE', 'BA', 'CAT', 'MMM', 'HON',
    # Energy
    'XOM', 'CVX', 'COP',
    # Telecom
    'T', 'VZ', 'TMUS',
    # Utilities
    'NEE', 'DUK', 'SO'
]

print(f"Using {len(tickers)} stocks for the analysis")

### 2.1 Traditional CAPM Approach

First, let's use the traditional CAPM approach to calculate residual returns by removing the market factor.

In [ ]:
# Fetch data and calculate returns, betas, and residual returns
try:
    start_time = time.time()
    returns, betas, residual_returns = prepare_stock_data(
        tickers, 
        start_date, 
        end_date, 
        beta_window=beta_window
    )
    print(f"Data preparation completed in {time.time() - start_time:.2f} seconds")
    
    # Check the dimensions of our data
    print(f"Returns shape: {returns.shape}")
    print(f"Betas shape: {betas.shape}")
    print(f"Residual returns shape: {residual_returns.shape}")
    
    # Check which tickers we actually have data for
    actual_tickers = returns.columns.drop('SPY').tolist()
    print(f"Actual number of stocks with data: {len(actual_tickers)}")
    
    missing_tickers = [ticker for ticker in tickers if ticker not in actual_tickers]
    if missing_tickers:
        print(f"Missing data for: {missing_tickers}")
    
except Exception as e:
    print(f"Error fetching data: {e}")
    
    # For demonstration purposes, create mock data if fetching fails
    print("Creating mock data for demonstration...")
    
    dates = pd.date_range(start=start_date, end=end_date, freq='B')
    mock_returns = pd.DataFrame(
        np.random.normal(0.0005, 0.015, (len(dates), len(tickers) + 1)),
        index=dates,
        columns=tickers + ['SPY']
    )
    
    mock_betas = pd.DataFrame(
        np.random.normal(1.0, 0.3, (len(dates) - beta_window, len(tickers))),
        index=dates[beta_window:],
        columns=tickers
    )
    
    mock_residual_returns = pd.DataFrame(
        np.random.normal(0.0, 0.01, (len(dates) - beta_window, len(tickers))),
        index=dates[beta_window:],
        columns=tickers
    )
    
    returns = mock_returns
    betas = mock_betas
    residual_returns = mock_residual_returns
    actual_tickers = tickers

### 2.2 Enhanced Multi-Factor Models

Now, let's use the enhanced multi-factor models approach to better isolate stock-specific returns.

In [ ]:
# Use Fama-French 5-factor model
start_time = time.time()
ff_returns, ff_factors, ff_residuals, ff_data = prepare_multifactor_data(
    actual_tickers, 
    start_date, 
    end_date, 
    factor_model='fama_french'
)
print(f"Fama-French model preparation completed in {time.time() - start_time:.2f} seconds")

# Use PCA-based factor model
start_time = time.time()
pca_returns, pca_factors, pca_residuals, pca_data = prepare_multifactor_data(
    actual_tickers, 
    start_date, 
    end_date, 
    factor_model='pca', 
    n_components=5
)
print(f"PCA model preparation completed in {time.time() - start_time:.2f} seconds")

# Compare the results - check correlation between CAPM and enhanced residuals
capm_pca_corr = pd.DataFrame(
    index=actual_tickers,
    columns=['CAPM_FF_Corr', 'CAPM_PCA_Corr', 'FF_PCA_Corr']
)

for ticker in actual_tickers:
    if ticker in ff_residuals.columns and ticker in pca_residuals.columns:
        capm_pca_corr.loc[ticker, 'CAPM_FF_Corr'] = residual_returns[ticker].corr(ff_residuals[ticker])
        capm_pca_corr.loc[ticker, 'CAPM_PCA_Corr'] = residual_returns[ticker].corr(pca_residuals[ticker])
        capm_pca_corr.loc[ticker, 'FF_PCA_Corr'] = ff_residuals[ticker].corr(pca_residuals[ticker])

print("\nCorrelation between different residualization methods:")
print(capm_pca_corr.describe())

# Visualize the distribution of correlations
fig = px.box(
    capm_pca_corr.melt(), 
    y="value", 
    x="variable", 
    title="Correlation Between Different Residualization Methods",
    labels={"value": "Correlation", "variable": "Methods Compared"}
)
fig.update_layout(yaxis_range=[-1, 1])
fig.show()

# For the rest of this notebook, we'll continue with the CAPM residuals for simplicity
# But in practice, you might want to use the enhanced methods for better results

In [ ]:
# Let's examine the distribution of betas and residual returns:

# Plot the distribution of betas for the most recent period
recent_betas = betas.iloc[-1]

fig = px.bar(
    x=recent_betas.index, 
    y=recent_betas.values,
    title='Current Beta Values by Stock',
    labels={'x': 'Stock', 'y': 'Beta Value'}
)
fig.add_hline(y=1.0, line_dash="dash", line_color="red")
fig.update_layout(xaxis_tickangle=-90)
fig.show()

# Plot the distribution of residual returns vs raw returns
# Get statistics for the most recent period (last 60 days)
recent_raw_returns = returns.iloc[-60:].drop('SPY', axis=1)
recent_residual_returns = residual_returns.iloc[-60:]

# Compute volatility (standard deviation)
raw_volatility = recent_raw_returns.std()
residual_volatility = recent_residual_returns.std()

# Create scatter plot
fig = px.scatter(
    x=raw_volatility, 
    y=residual_volatility,
    text=raw_volatility.index,
    title='Raw Return vs Residual Return Volatility',
    labels={'x': 'Raw Return Volatility', 'y': 'Residual Return Volatility'}
)
fig.update_traces(textposition='top center')
fig.show()

# Volatility reduction after removing market factor
volatility_reduction = (1 - residual_volatility / raw_volatility) * 100
avg_reduction = volatility_reduction.mean()

print(f"Average volatility reduction after removing market factor: {avg_reduction:.2f}%")

## 3. Correlation Matrix Analysis

Next, we'll compute and analyze the correlation matrix of residual returns.

In [ ]:
# Compute correlation matrix using the last 60 days of residual returns
correlation_window = 60
correlation_matrix = compute_correlation_matrix(residual_returns.iloc[-correlation_window:])

# Visualize the correlation matrix using Plotly for interactivity
fig = px.imshow(
    correlation_matrix,
    title='Residual Returns Correlation Matrix',
    labels=dict(x="Stock", y="Stock", color="Correlation"),
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1
)
fig.update_layout(
    width=800,
    height=800,
    xaxis_tickangle=-45
)
fig.show()

# Decompose the correlation matrix into positive and negative components
positive_matrix, negative_matrix = decompose_correlation_matrix(correlation_matrix)

# Visualize the positive matrix
fig = px.imshow(
    positive_matrix,
    title='Positive Correlations',
    labels=dict(x="Stock", y="Stock", color="Correlation"),
    x=positive_matrix.columns,
    y=positive_matrix.columns,
    color_continuous_scale='Blues',
    zmin=0, zmax=1
)
fig.update_layout(
    width=800,
    height=800,
    xaxis_tickangle=-45
)
fig.show()

# Visualize the negative matrix (absolute values)
fig = px.imshow(
    negative_matrix,
    title='Negative Correlations (Absolute Values)',
    labels=dict(x="Stock", y="Stock", color="Correlation"),
    x=negative_matrix.columns,
    y=negative_matrix.columns,
    color_continuous_scale='Reds',
    zmin=0, zmax=1
)
fig.update_layout(
    width=800,
    height=800,
    xaxis_tickangle=-45
)
fig.show()

Let's analyze the eigenvalue distribution to determine the optimal number of clusters:

In [ ]:
# Calculate the number of clusters using different methods
n_clusters_mp = determine_clusters_mp(correlation_matrix, correlation_window)
n_clusters_var = determine_clusters_variance(correlation_matrix, variance_explained=0.9)

print(f"Number of clusters (Marchenko-Pastur): {n_clusters_mp}")
print(f"Number of clusters (90% Variance): {n_clusters_var}")

# Analyze eigenvalue distribution
eigenvalues, variance_ratios = get_eigenvalue_distribution(correlation_matrix)

# Eigenvalue distribution plot using Plotly
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Eigenvalue Distribution', 'Cumulative Variance Explained')
)

fig.add_trace(
    go.Scatter(
        x=list(range(1, len(eigenvalues) + 1)),
        y=eigenvalues,
        mode='lines+markers',
        name='Eigenvalues'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=list(range(1, len(variance_ratios) + 1)),
        y=variance_ratios,
        mode='lines+markers',
        name='Cumulative Variance'
    ),
    row=1, col=2
)

# Add line for 90% variance explained
fig.add_shape(
    type='line',
    x0=0,
    y0=0.9,
    x1=len(variance_ratios),
    y1=0.9,
    line=dict(color='red', dash='dash'),
    row=1, col=2
)

fig.update_layout(
    height=400,
    width=900,
    title_text='Eigenvalue Analysis for Cluster Determination'
)
fig.update_xaxes(title_text='Eigenvalue Number', row=1, col=1)
fig.update_xaxes(title_text='Number of Eigenvalues', row=1, col=2)
fig.update_yaxes(title_text='Eigenvalue', row=1, col=1)
fig.update_yaxes(title_text='Cumulative Variance Ratio', row=1, col=2)
fig.show()

## 4. Clustering

### 4.1 Basic Clustering Methods

First, let's apply the basic clustering methods from the original paper:

In [ ]:
# Apply different clustering methods
clustering_methods = ['spectral', 'signed_laplacian_sym', 'signed_laplacian_rw', 'sponge', 'sponge_sym']
clustering_results = {}

# Use the MP method for determining number of clusters
n_clusters = n_clusters_mp

for method in clustering_methods:
    # Cluster the stocks
    labels, actual_n_clusters = cluster_stocks(
        correlation_matrix, 
        method=method, 
        n_clusters=n_clusters
    )
    
    # Get clusters as dictionary
    clusters = get_clusters_dict(labels, correlation_matrix.columns.tolist())
    
    # Store results
    clustering_results[method] = {
        'labels': labels,
        'n_clusters': actual_n_clusters,
        'clusters': clusters
    }
    
    print(f"\n{method.upper()} clustering with {actual_n_clusters} clusters:")
    for cluster_id, cluster_tickers in clusters.items():
        print(f"Cluster {cluster_id} ({len(cluster_tickers)} stocks): {', '.join(cluster_tickers)}")

### 4.2 Enhanced Clustering Methods

Now, let's try the enhanced clustering methods:

In [ ]:
# Apply advanced clustering methods
advanced_methods = ['hierarchical', 'density']
advanced_results = {}

for method in advanced_methods:
    # Set parameters based on method
    kwargs = {}
    if method == 'hierarchical':
        kwargs = {'method': 'ward'}  # Use Ward's method for hierarchical
    elif method == 'density':
        kwargs = {'min_cluster_size': 3}  # Minimum cluster size for density clustering
    
    # Cluster the stocks
    labels, actual_n_clusters = cluster_stocks(
        correlation_matrix, 
        method=method, 
        n_clusters=n_clusters,
        **kwargs
    )
    
    # Get clusters as dictionary
    clusters = get_clusters_dict(labels, correlation_matrix.columns.tolist())
    
    # Store results
    advanced_results[method] = {
        'labels': labels,
        'n_clusters': actual_n_clusters,
        'clusters': clusters
    }
    
    print(f"\n{method.upper()} clustering with {actual_n_clusters} clusters:")
    for cluster_id, cluster_tickers in clusters.items():
        print(f"Cluster {cluster_id} ({len(cluster_tickers)} stocks): {', '.join(cluster_tickers)}")

# Let's try deep embedding clustering on a subset of residual returns (for speed)
try:
    # Use a subset of data for deep embedding
    subset_returns = residual_returns.iloc[-100:]
    
    deep_labels, embeddings, encoder = deep_embedding_clustering(
        subset_returns, 
        n_clusters=5,  # Fewer clusters for simplicity
        embedding_dim=10, 
        epochs=50
    )
    
    deep_clusters = get_clusters_dict(deep_labels, subset_returns.columns.tolist())
    
    print("\nDEEP EMBEDDING clustering:")
    for cluster_id, cluster_tickers in deep_clusters.items():
        print(f"Cluster {cluster_id} ({len(cluster_tickers)} stocks): {', '.join(cluster_tickers)}")
        
    # Visualize the embeddings
    if embeddings is not None and embeddings.shape[1] >= 2:
        fig = px.scatter(
            x=embeddings[:, 0],
            y=embeddings[:, 1],
            color=[str(label) for label in deep_labels],
            text=subset_returns.columns,
            title='Deep Embedding Clustering (2D Projection)',
            labels={'x': 'Dimension 1', 'y': 'Dimension 2', 'color': 'Cluster'}
        )
        fig.update_traces(textposition='top center')
        fig.show()

except Exception as e:
    print(f"Error with deep embedding clustering: {e}")
    print("Skipping deep embedding visualization")

### 4.3 Visualize Clustering Results

Visualize the SPONGE clustering result with clusters highlighted:

In [ ]:
# Visualize clusters for the SPONGE algorithm
sponge_labels = clustering_results['sponge_sym']['labels']

# Calculate silhouette score to evaluate clustering quality
from sklearn.metrics import silhouette_score
sil_score = silhouette_score(
    1 - np.abs(correlation_matrix.values),  # Convert correlation to distance
    sponge_labels,
    metric='precomputed'
)
print(f"Silhouette Score for SPONGE_SYM clustering: {sil_score:.4f}")

# Get order of stocks sorted by cluster
sorted_indices = np.argsort(sponge_labels)
sorted_tickers = [correlation_matrix.columns[i] for i in sorted_indices]

# Reorder the correlation matrix
sorted_corr = correlation_matrix.loc[sorted_tickers, sorted_tickers]

# Create a heatmap with cluster boundaries
fig = px.imshow(
    sorted_corr,
    title='Correlation Matrix Sorted by SPONGE_SYM Clusters',
    labels=dict(x="Stock", y="Stock", color="Correlation"),
    x=sorted_tickers,
    y=sorted_tickers,
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1
)

# Add cluster boundaries
cluster_boundaries = []
current_cluster = sponge_labels[sorted_indices[0]]

for i, idx in enumerate(sorted_indices[1:], 1):
    if sponge_labels[idx] != current_cluster:
        # Add vertical line
        fig.add_shape(
            type="line",
            x0=i-0.5,
            x1=i-0.5,
            y0=-0.5,
            y1=len(sorted_tickers)-0.5,
            line=dict(color="black", width=2)
        )
        
        # Add horizontal line
        fig.add_shape(
            type="line",
            x0=-0.5,
            x1=len(sorted_tickers)-0.5,
            y0=i-0.5,
            y1=i-0.5,
            line=dict(color="black", width=2)
        )
        
        current_cluster = sponge_labels[idx]

fig.update_layout(
    width=800,
    height=800,
    xaxis_tickangle=-45
)
fig.show()

## 5. Adaptive Parameter Management

Let's demonstrate how we can use adaptive parameters based on market regimes:

In [ ]:
# Initialize adaptive parameter manager
adaptive_manager = AdaptiveParameterManager(
    returns,
    lookback_min=3,
    lookback_max=20,
    rebalance_min=1,
    rebalance_max=7,
    threshold_min=0.0,
    threshold_max=0.01,
    correlation_window_min=10,
    correlation_window_max=60,
    n_regimes=3
)

# Visualize detected market regimes
adaptive_manager.visualize_regimes()

# Get optimal parameters for different methods
optimal_lookback_vol = adaptive_manager.calculate_optimal_lookback(method='volatility')
optimal_lookback_acf = adaptive_manager.calculate_optimal_lookback(method='autocorrelation')
optimal_lookback_reg = adaptive_manager.calculate_optimal_lookback(method='regime')

optimal_rebalance = adaptive_manager.calculate_optimal_rebalance_period(method='volatility')
optimal_threshold = adaptive_manager.calculate_optimal_threshold(method='volatility')
optimal_corr_window = adaptive_manager.calculate_optimal_correlation_window(method='volatility')

print("\nOptimal Parameters based on current market conditions:")
print(f"Lookback Window (Volatility): {optimal_lookback_vol} days")
print(f"Lookback Window (Autocorrelation): {optimal_lookback_acf} days")
print(f"Lookback Window (Regime): {optimal_lookback_reg} days")
print(f"Rebalance Period: {optimal_rebalance} days")
print(f"Threshold: {optimal_threshold:.6f}")
print(f"Correlation Window: {optimal_corr_window} days")

# Get regime-specific parameters
regime_params = adaptive_manager.get_regime_specific_parameters()

print("\nRegime-Specific Parameters:")
for key, value in regime_params.items():
    if isinstance(value, float):
        print(f"{key}: {value:.6f}")
    else:
        print(f"{key}: {value}")

## 6. Portfolio Construction

With our clusters defined, we'll now construct mean-reverting portfolios within each cluster.

In [ ]:
# Use the SPONGE clustering method for portfolio construction
chosen_method = 'sponge_sym'
clusters = clustering_results[chosen_method]['clusters']

# Set portfolio parameters (or use adaptive parameters)
lookback_window = optimal_lookback_vol  # Use adaptive parameter
threshold = optimal_threshold  # Use adaptive parameter
holding_period = 30  # Days to hold the portfolio

# Identify winners and losers
winners, losers = identify_winners_losers(
    returns.iloc[-lookback_window:],
    clusters,
    lookback_window,
    threshold
)

# Print winners and losers
print(f"\nWinners and Losers for {chosen_method.upper()} clusters:")
for cluster_id in clusters.keys():
    cluster_winners = winners.get(cluster_id, [])
    cluster_losers = losers.get(cluster_id, [])
    
    print(f"\nCluster {cluster_id}:")
    print(f"  Winners ({len(cluster_winners)}): {', '.join(cluster_winners)}")
    print(f"  Losers ({len(cluster_losers)}): {', '.join(cluster_losers)}")

# Construct arbitrage portfolios
portfolios = construct_arbitrage_portfolios(winners, losers)

# Print portfolio compositions
print("\nPortfolio Compositions:")
for cluster_id, portfolio in portfolios.items():
    print(f"\nCluster {cluster_id}:")
    for ticker, weight in portfolio.items():
        position = "LONG" if weight > 0 else "SHORT"
        print(f"  {ticker}: {position} {abs(weight):.4f}")

## 7. Alternative Data Integration

Let's demonstrate how alternative data can be integrated into the portfolio construction process:

In [ ]:
# Initialize data processors
sentiment_processor = SentimentDataProcessor()
options_processor = OptionsDataProcessor()
flow_processor = OrderFlowProcessor()

# Generate mock data for demonstration
print("Generating mock alternative data for demonstration...")

# 1. Sentiment data
sentiment_data = sentiment_processor.fetch_news_sentiment(
    actual_tickers, 
    returns.index[-60].strftime('%Y-%m-%d'), 
    returns.index[-1].strftime('%Y-%m-%d'),
    provider='mock'
)

social_sentiment = sentiment_processor.fetch_social_sentiment(
    actual_tickers, 
    returns.index[-60].strftime('%Y-%m-%d'), 
    returns.index[-1].strftime('%Y-%m-%d'),
    provider='mock'
)

# Aggregate sentiment from different sources
aggregated_sentiment = sentiment_processor.aggregate_sentiment(
    social_sentiment,
    method='exponential_decay',
    decay_factor=0.9
)

print("\nSentiment Data Sample:")
print(aggregated_sentiment.head())

# Visualize sentiment over time for a few stocks
sample_stocks = actual_tickers[:5]
fig = go.Figure()

for ticker in sample_stocks:
    fig.add_trace(go.Scatter(
        x=aggregated_sentiment.index,
        y=aggregated_sentiment[ticker],
        mode='lines',
        name=ticker
    ))

fig.update_layout(
    title='Sentiment Scores Over Time',
    xaxis_title='Date',
    yaxis_title='Sentiment Score',
    yaxis_range=[-1, 1],
    legend_title='Stock'
)
fig.show()

# 2. Options data
options_data = options_processor.fetch_options_data(
    actual_tickers[:5],  # Just use a few stocks for speed
    returns.index[-60].strftime('%Y-%m-%d'), 
    returns.index[-1].strftime('%Y-%m-%d'),
    provider='mock'
)

# Extract options signals
options_signals = {}
for ticker in actual_tickers[:5]:
    if ticker in options_data:
        options_signals[ticker] = options_processor.extract_options_signals(
            options_data, ticker
        )

print("\nOptions Signals Sample:")
for ticker, signals in list(options_signals.items())[:2]:
    print(f"{ticker}:")
    for signal, value in signals.items():
        print(f"  {signal}: {value:.4f}")

# 3. Order flow data
order_flow_data = flow_processor.fetch_order_flow_data(
    actual_tickers[:5],
    returns.index[-60].strftime('%Y-%m-%d'), 
    returns.index[-1].strftime('%Y-%m-%d'),
    provider='mock'
)

# Calculate order flow imbalance
order_flow_imbalance = {}
for ticker, data in order_flow_data.items():
    order_flow_imbalance[ticker] = flow_processor.calculate_order_flow_imbalance(
        data, resample_freq='1d'
    )

print("\nOrder Flow Imbalance Sample:")
for ticker, imbalance in list(order_flow_imbalance.items())[:2]:
    print(f"{ticker}:")
    print(imbalance.head())

# Apply alternative data to portfolio construction
# 1. Adjust returns using sentiment
sentiment_adjusted_returns = returns.iloc[-lookback_window:].copy()
for date in sentiment_adjusted_returns.index:
    if date in aggregated_sentiment.index:
        for ticker in sentiment_adjusted_returns.columns:
            if ticker in aggregated_sentiment.columns:
                sentiment_score = aggregated_sentiment.loc[date, ticker]
                sentiment_adjusted_returns.loc[date, ticker] *= (1 + sentiment_score * 0.1)

# 2. Identify winners and losers with sentiment-adjusted returns
sentiment_winners, sentiment_losers = identify_winners_losers(
    sentiment_adjusted_returns,
    clusters,
    lookback_window,
    threshold
)

# 3. Apply options signals
for ticker, signals in options_signals.items():
    for cluster_id in winners.keys():
        # If high put/call ratio, move from winners to losers
        if ticker in sentiment_winners.get(cluster_id, []) and signals.get('put_call_ratio', 1.0) > 1.3:
            sentiment_winners[cluster_id].remove(ticker)
            if ticker not in sentiment_losers[cluster_id]:
                sentiment_losers[cluster_id].append(ticker)
            print(f"Moved {ticker} from winners to losers due to high put/call ratio")
            
        # If very negative skew, move from losers to winners
        elif ticker in sentiment_losers.get(cluster_id, []) and signals.get('skew_spread', 0) < -10:
            sentiment_losers[cluster_id].remove(ticker)
            if ticker not in sentiment_winners[cluster_id]:
                sentiment_winners[cluster_id].append(ticker)
            print(f"Moved {ticker} from losers to winners due to negative skew")

# Construct enhanced portfolios
enhanced_portfolios = construct_arbitrage_portfolios(sentiment_winners, sentiment_losers)

print("\nEnhanced Portfolio Compositions (with Alternative Data):")
for cluster_id, portfolio in enhanced_portfolios.items():
    print(f"\nCluster {cluster_id}:")
    for ticker, weight in portfolio.items():
        position = "LONG" if weight > 0 else "SHORT"
        print(f"  {ticker}: {position} {abs(weight):.4f}")

## 8. Portfolio Evaluation

We'll evaluate the performance of both standard and enhanced portfolios.

In [ ]:
# Define evaluation period (last 30 days of data)
evaluation_period = 30
evaluation_returns = returns.iloc[-evaluation_period:]

# Calculate standard portfolio returns
standard_portfolio_returns = calculate_portfolio_returns(
    evaluation_returns, portfolios
)

# Calculate enhanced portfolio returns
enhanced_portfolio_returns = calculate_portfolio_returns(
    evaluation_returns, enhanced_portfolios
)

# Display portfolio returns statistics
print("\nStandard Portfolio Returns Statistics:")
print(f"Mean Daily Return: {standard_portfolio_returns['Combined'].mean():.4%}")
print(f"Standard Deviation: {standard_portfolio_returns['Combined'].std():.4%}")
print(f"Sharpe Ratio: {calculate_sharpe_ratio(standard_portfolio_returns['Combined']):.4f}")
print(f"Sortino Ratio: {calculate_sortino_ratio(standard_portfolio_returns['Combined']):.4f}")
print(f"Cumulative Return: {(1 + standard_portfolio_returns['Combined']).prod() - 1:.4%}")
print(f"Annualized Return: {calculate_annualized_return(standard_portfolio_returns['Combined']):.4%}")
print(f"Win Rate: {(standard_portfolio_returns['Combined'] > 0).mean():.4%}")

print("\nEnhanced Portfolio Returns Statistics:")
print(f"Mean Daily Return: {enhanced_portfolio_returns['Combined'].mean():.4%}")
print(f"Standard Deviation: {enhanced_portfolio_returns['Combined'].std():.4%}")
print(f"Sharpe Ratio: {calculate_sharpe_ratio(enhanced_portfolio_returns['Combined']):.4f}")
print(f"Sortino Ratio: {calculate_sortino_ratio(enhanced_portfolio_returns['Combined']):.4f}")
print(f"Cumulative Return: {(1 + enhanced_portfolio_returns['Combined']).prod() - 1:.4%}")
print(f"Annualized Return: {calculate_annualized_return(enhanced_portfolio_returns['Combined']):.4%}")
print(f"Win Rate: {(enhanced_portfolio_returns['Combined'] > 0).mean():.4%}")

# Calculate cumulative returns
std_cumulative = (1 + standard_portfolio_returns['Combined']).cumprod() - 1
enhanced_cumulative = (1 + enhanced_portfolio_returns['Combined']).cumprod() - 1

# Plot cumulative returns comparison
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=std_cumulative.index,
    y=std_cumulative,
    mode='lines',
    name='Standard Portfolio'
))
fig.add_trace(go.Scatter(
    x=enhanced_cumulative.index,
    y=enhanced_cumulative,
    mode='lines',
    name='Enhanced Portfolio'
))
fig.update_layout(
    title='Cumulative Portfolio Returns',
    xaxis_title='Date',
    yaxis_title='Return',
    yaxis_tickformat='.1%',
    legend_title='Portfolio Type'
)
fig.show()

## 9. Advanced Backtesting

Let's demonstrate the advanced backtesting features:

In [ ]:
# Initialize a backtest instance for the full period
backtest = StatisticalArbitrageBacktest(
    tickers=actual_tickers,
    start_date=start_date,
    end_date=end_date,
    clustering_method='sponge_sym',
    beta_window=60,
    correlation_window=20,
    lookback_window=5,
    rebalance_period=3,
    threshold=0.0,
    stop_win_threshold=0.05
)

# Run the backtest
print("Running full backtest...")
portfolio_returns = backtest.run_backtest()
metrics = backtest.get_performance_metrics()

print("\nFull Backtest Metrics:")
print(metrics.loc['Combined'])

# Get market benchmark returns (SPY)
spy_returns = returns['SPY']

# Plot cumulative returns comparison with benchmark
plot_cumulative_returns(
    {'SPONGE Clustering': portfolio_returns},
    spy_returns,
    title='Cumulative Returns: Strategy vs Market'
)

# Plot drawdowns
plot_drawdowns(
    {'SPONGE Clustering': portfolio_returns},
    title='Strategy Drawdowns'
)

# Plot monthly returns heatmap
plot_monthly_returns_heatmap(
    {'SPONGE Clustering': portfolio_returns},
    'SPONGE Clustering',
    title='Monthly Returns: SPONGE Clustering'
)

# Time-Series Cross-Validation (simplified for speed)
print("\nRunning Time-Series Cross-Validation (simplified)...")
# Note: This would typically take longer, so we're using a smaller subset and fewer splits
tscv_results = time_series_cross_validation(
    actual_tickers[:15],  # Use fewer tickers for speed
    start_date,
    end_date,
    n_splits=2,  # Fewer splits for example
    backtest_params={
        'clustering_method': 'spectral',  # Faster method
        'lookback_window': 5,
        'rebalance_period': 3,
        'threshold': 0.0
    }
)

print("\nTime-Series CV Results:")
if 'mean_sharpe' in tscv_results:
    print(f"Mean Sharpe: {tscv_results['mean_sharpe']:.4f} ± {tscv_results['std_sharpe']:.4f}")
    print(f"Mean Sortino: {tscv_results['mean_sortino']:.4f} ± {tscv_results['std_sortino']:.4f}")
    print(f"Mean Annualized Return: {tscv_results['mean_annualized_return']:.4%} ± {tscv_results['std_annualized_return']:.4%}")
else:
    print("Cross-validation did not complete successfully. Check logs for details.")

# Monte Carlo Simulation (simplified for speed)
print("\nRunning Monte Carlo Simulation (simplified)...")
mc_results = monte_carlo_backtest(
    returns.iloc[-252:],  # Use only last year for speed
    residual_returns.iloc[-252:],  # Use only last year for speed
    n_simulations=50,  # Fewer simulations for example
    backtest_params={
        'clustering_method': 'spectral',  # Faster method
        'lookback_window': 5,
        'rebalance_period': 3,
        'threshold': 0.0
    },
    tickers=actual_tickers[:15]  # Use fewer tickers for speed
)

print("\nMonte Carlo Results:")
if 'mean' in mc_results:
    print(f"Mean Sharpe: {mc_results['mean']['sharpe_ratio']:.4f} ± {mc_results['std']['sharpe_ratio']:.4f}")
    print(f"5th-95th Percentile Sharpe: [{mc_results['5th_percentile']['sharpe_ratio']:.4f}, {mc_results['95th_percentile']['sharpe_ratio']:.4f}]")
    print(f"Mean Annualized Return: {mc_results['mean']['annualized_return']:.4%} ± {mc_results['std']['annualized_return']:.4%}")
else:
    print("Monte Carlo simulation did not complete successfully. Check logs for details.")

# Walk-Forward Optimization (simplified for speed)
print("\nRunning Walk-Forward Optimization (simplified)...")
wf_results = walk_forward_optimization(
    actual_tickers[:15],  # Use fewer tickers for speed
    start_date,
    end_date,
    initial_window=252,
    step=63,  # Quarterly steps
    param_grid={
        'lookback_window': [3, 10],  # Simplified grid
        'rebalance_period': [1, 5],
        'threshold': [0.0, 0.005]
    },
    clustering_method='spectral'  # Faster method
)

print("\nWalk-Forward Optimization Results:")
if 'weighted_sharpe' in wf_results:
    print(f"Weighted Sharpe: {wf_results['weighted_sharpe']:.4f}")
    print(f"Weighted Annualized Return: {wf_results['weighted_annual_return']:.4%}")

    print("\nParameter Frequency:")
    for param, freq in wf_results['parameter_frequency'].items():
        print(f"{param}:", end=" ")
        sorted_freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
        for value, count in sorted_freq:
            print(f"{value}: {count}", end=", ")
        print()
else:
    print("Walk-forward optimization did not complete successfully. Check logs for details.")

## 10. Comparison to Industry Benchmark

Let's compare our clustering approach to a traditional industry-based approach:

In [ ]:
# Get Fama-French 12 industry classifications
industry_mapping = get_fama_french_industries(actual_tickers)
industry_names = get_industry_names()

# Print industry classifications
print("\nFama-French 12 Industry Classifications:")
for ticker, industry_id in sorted(industry_mapping.items()):
    industry_name = industry_names.get(industry_id, "Unknown")
    print(f"{ticker}: {industry_id} - {industry_name}")

# Run industry benchmark
print("\nRunning industry-based benchmark...")
industry_returns, industry_metrics = run_industry_benchmark(
    tickers=actual_tickers,
    start_date=start_date,
    end_date=end_date,
    industry_mapping=industry_mapping,
    beta_window=60,
    correlation_window=20,
    lookback_window=5,
    rebalance_period=3,
    threshold=0.0,
    stop_win_threshold=0.05
)

print("\nIndustry Benchmark Metrics:")
print(industry_metrics.loc['Combined'])

# Compare cumulative returns
clustering_cumulative = (1 + portfolio_returns['Combined']).cumprod() - 1
industry_cumulative = (1 + industry_returns['Combined']).cumprod() - 1
spy_cumulative = (1 + spy_returns[clustering_cumulative.index[0]:clustering_cumulative.index[-1]]).cumprod() - 1

# Plot comparison
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=clustering_cumulative.index,
    y=clustering_cumulative,
    mode='lines',
    name='Clustering Approach'
))
fig.add_trace(go.Scatter(
    x=industry_cumulative.index,
    y=industry_cumulative,
    mode='lines',
    name='Industry Benchmark'
))
fig.add_trace(go.Scatter(
    x=spy_cumulative.index,
    y=spy_cumulative,
    mode='lines',
    name='S&P 500 (SPY)',
    line=dict(dash='dash')
))
fig.update_layout(
    title='Cumulative Returns Comparison',
    xaxis_title='Date',
    yaxis_title='Return',
    yaxis_tickformat='.1%',
    legend_title='Strategy'
)
fig.show()

# Compare key metrics
comparison_df = pd.DataFrame({
    'Clustering': metrics.loc['Combined', ['Annualized Return (%)', 'Sharpe Ratio', 'Sortino Ratio', 'Maximum Drawdown (%)']],
    'Industry': industry_metrics.loc['Combined', ['Annualized Return (%)', 'Sharpe Ratio', 'Sortino Ratio', 'Maximum Drawdown (%)']]
})

print("\nMetrics Comparison:")
print(comparison_df)

fig = go.Figure(data=[
    go.Bar(name='Clustering', x=comparison_df.index, y=comparison_df['Clustering']),
    go.Bar(name='Industry', x=comparison_df.index, y=comparison_df['Industry'])
])
fig.update_layout(
    barmode='group',
    title='Performance Metrics Comparison',
    xaxis_title='Metric',
    yaxis_title='Value'
)
fig.show()

## 11. Portfolio Integration

Finally, let's demonstrate how to integrate our statistical arbitrage strategy with existing portfolios:

In [ ]:
# Create mock strategy returns for portfolio integration
# S&P 500
spy_strategy = spy_returns

# Value strategy (higher returns during value regimes)
dates = returns.index
value_returns = pd.Series(index=dates, dtype=float)
for i in range(len(dates)):
    # Add some mean-reversion and value-growth cycle
    cycle = np.sin(i / 180 * np.pi)  # Roughly 1.5 year value-growth cycle
    # More positive during value regimes, negative during growth regimes
    value_returns[i] = 0.0003 + 0.001 * cycle + np.random.normal(0, 0.008)

# Momentum strategy
momentum_returns = pd.Series(index=dates, dtype=float)
for i in range(len(dates)):
    # Add trend persistence
    if i > 0:
        momentum_returns[i] = 0.0004 + 0.2 * momentum_returns[i-1] + np.random.normal(0, 0.009)
    else:
        momentum_returns[i] = 0.0004 + np.random.normal(0, 0.009)

# Use our statistical arbitrage strategy
stat_arb_returns = portfolio_returns['Combined']

# Create dictionary of strategies
strategies = {
    'S&P 500': spy_strategy,
    'Value': value_returns,
    'Momentum': momentum_returns,
    'StatArb': stat_arb_returns
}

# Make sure all strategies have the same date range
common_dates = stat_arb_returns.index
for name in strategies:
    strategies[name] = strategies[name].loc[strategies[name].index.isin(common_dates)]

# Initialize portfolio allocator
allocator = PortfolioAllocator(strategies, risk_free_rate=0.005)

# Print strategy metrics
print("\nStrategy Metrics:")
print(allocator.metrics)

# Print correlation matrix
print("\nCorrelation Matrix:")
print(allocator.correlation_matrix)

# Try different allocation methods
for method in ['risk_parity', 'min_variance', 'max_diversification', 'equal_weight']:
    weights = allocator.optimize_allocation(method)
    print(f"\n{method.replace('_', ' ').title()} Weights:")
    print(weights)
    
    metrics = allocator.calculate_portfolio_metrics(weights)
    print(f"\n{method.replace('_', ' ').title()} Portfolio Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")

# Cluster strategies to find natural groupings
clustering = allocator.cluster_strategies()
print("\nStrategy Clusters:")
for cluster_id, strategies in clustering['clusters'].items():
    print(f"Cluster {cluster_id}: {', '.join(strategies)}")

# Calculate impact of adding statistical arbitrage to a portfolio of SPY
impact = calculate_information_ratio_impact(
    spy_strategy,
    stat_arb_returns,
    allocation=0.15
)

print("\nImpact of Adding Statistical Arbitrage to S&P 500:")
for key, value in impact.items():
    print(f"{key}: {value:.4f}")

# Visualize the efficient frontier
allocator.plot_efficient_frontier(n_portfolios=1000)

## 12. Conclusion

In this notebook, we've implemented the correlation matrix clustering approach for statistical arbitrage from Cartea, Cucuringu, and Jin (2023), along with significant enhancements:

1. **Advanced Factor Models**: Beyond CAPM, we've explored Fama-French and PCA-based factor models to better isolate stock-specific returns.

2. **Enhanced Clustering Methods**: In addition to the original methods, we've implemented hierarchical, density-based, and deep embedding clustering approaches.

3. **Adaptive Parameter Management**: We've demonstrated how to dynamically adjust strategy parameters based on market regimes.

4. **Alternative Data Integration**: We've shown how sentiment, options, and order flow data can enhance clustering and signal generation.

5. **Advanced Backtesting**: We've demonstrated time-series cross-validation, Monte Carlo simulation, and walk-forward optimization for robust performance evaluation.

6. **Portfolio Integration**: We've shown how to optimally combine the statistical arbitrage strategy with existing portfolios.

The results show that clustering-based statistical arbitrage can identify temporary mispricings within groups of similar stocks, and our enhancements can further improve strategy performance and robustness.